In [1]:
import pandas as pd
import numpy as np

#### Nesa etapa iremos fazer a definição de tipos, bem como o tratamentos dos dados faltantes,normalização, discretização e limpeza do dados. Todo o processo de pré processamento.

In [26]:
###Essa base é a base seed, através dos ids dos jogadores, iremos fazer os joins até chegar
###na base final utilizada nessa etapa
df_users_seed = pd.read_csv("SummonerFinalData.csv", index_col = 'accountId')

### Lista das 10 ultimas partidas de cada jogador na base seed. 
df_matches_acc_id = pd.read_csv("matchByAccounbtIdFiltered10.csv", index_col = 'accountId')

In [27]:
####Print de algumas informações basicas######
print(df_users_seed.shape)
print(df_users_seed.dtypes)

(2661, 13)
Unnamed: 0        int64
tier             object
summonerId       object
summonerName     object
leaguePoints      int64
wins              int64
losses            int64
veteran            bool
inactive           bool
freshBlood         bool
hotStreak          bool
summonerLevel     int64
puuid            object
dtype: object


In [17]:
###Checar com professor a questão de tipagem.

In [29]:
####Dropando essa coluna intrusa
df_users_seed.drop('Unnamed: 0', axis = 1, inplace=True)

In [39]:
### Dados faltantes
getMissData(df_users_seed)

,names,types,NA #,NA %
tier,tier,object,0,0.0
summonerId,summonerId,object,0,0.0
summonerName,summonerName,object,0,0.0
leaguePoints,leaguePoints,int64,0,0.0
wins,wins,int64,0,0.0
losses,losses,int64,0,0.0
veteran,veteran,bool,0,0.0
inactive,inactive,bool,0,0.0
freshBlood,freshBlood,bool,0,0.0
hotStreak,hotStreak,bool,0,0.0


In [40]:
### Como os dados foram obtidos através de api, não temos dados faltantes

In [30]:
df_matches_acc_id.columns

Index(['platformId', 'gameId', 'champion', 'queue', 'season', 'timestamp',
       'role', 'lane'],
      dtype='object')

In [32]:
df_matches_acc_id.head(2)

,platformId,gameId,champion,queue,season,timestamp,role,lane
accountId,,,,,,,,
bk1m2_GS-2L9loYOKil6EbsEHnQOCY15LRVByLuQvjf46Cc,BR1,2071507283,114,440,13,1601166961222,SOLO,TOP
bk1m2_GS-2L9loYOKil6EbsEHnQOCY15LRVByLuQvjf46Cc,BR1,2071513192,236,440,13,1601164763566,DUO,TOP


In [41]:
### Dados faltantes
getMissData(df_matches_acc_id)

,names,types,NA #,NA %
platformId,platformId,object,0,0.0
gameId,gameId,int64,0,0.0
champion,champion,int64,0,0.0
queue,queue,int64,0,0.0
season,season,int64,0,0.0
timestamp,timestamp,int64,0,0.0
role,role,object,0,0.0
lane,lane,object,0,0.0


In [38]:
def getMissData(df):
     return pd.DataFrame({'names' : df.columns, 'types' : df.dtypes, 'NA #': df.isna().sum(), 'NA %' : (df.isna().sum() / df.shape[0]) * 100})
    